Using the Telco dataset

    Use the function defined in acquire.py to load the Telco data.

    Drop any unnecessary, unhelpful, or duplicated columns. This could mean dropping foreign key columns but keeping the corresponding string values, for example.

    Encode the categorical columns. Create dummy variables of the categorical columns and concatenate them onto the dataframe.

    Create a function named prep_telco that accepts the raw telco data, and returns the data with the transformations above applied.


In [1]:
import numpy as np
import pandas as pd
import acquire2

In [2]:
df = acquire2.get_telco_data()
df.head()

Using cached csv


,payment_type_id,contract_type_id,internet_service_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,contract_type,payment_type
0,2,2,1,0002-ORFBO,Female,0,Yes,Yes,9,Yes,...,Yes,Yes,No,Yes,65.6,593.3,No,DSL,One year,Mailed check
1,2,1,1,0003-MKNFE,Male,0,No,No,9,Yes,...,No,No,Yes,No,59.9,542.4,No,DSL,Month-to-month,Mailed check
2,1,1,2,0004-TLHLJ,Male,0,No,No,4,Yes,...,No,No,No,Yes,73.9,280.85,Yes,Fiber optic,Month-to-month,Electronic check
3,1,1,2,0011-IGKFF,Male,1,Yes,No,13,Yes,...,No,Yes,Yes,Yes,98.0,1237.85,Yes,Fiber optic,Month-to-month,Electronic check
4,2,1,2,0013-EXCHZ,Female,1,Yes,No,3,Yes,...,Yes,Yes,No,Yes,83.9,267.4,Yes,Fiber optic,Month-to-month,Mailed check


In [ ]:
def prep_telco_data(df):
    
    # Drop duplicate/unhelpful columns
    
    df.drop(columns = ['payment_type_id', 'internet_service_type_id', 'contract_type_id'], inplace=True)
       
    # Drop null values stored as whitespace
    
    df['total_charges'] = df['total_charges'].str.strip()
    df = df[df.total_charges != '']
    
    # Convert total charges to float
    
    df['total_charges'] = df.total_charges.astype(float)

    #convert senior_citizens into object
   
    df['senior_citizen'] = df.senior_citizen.map({ 1 : 'Yes', 0 : 'No'})#.astype(str)
   
    #use customer_id as index
    
    df.set_index('customer_id', inplace = True)
   
    
    # use replace to reduce multicolinearity for redundant values
    
    df.replace({'No phone service': 'No',
                'No internet service': 'No'},
                inplace = True)

    #remove : ' (automatic)'' from payment_type to make eventual plot labels easier since credit card and bank transfer are only auto
    df['payment_type'] = df['payment_type'].str.replace(' (automatic)', '', regex = False)

    # Convert binary categorical variables to numeric
    df['gender_encoded'] = df.gender.map({'Female': 1, 'Male': 0}).astype(int)
    df['partner_encoded'] = df.partner.map({'Yes': 1, 'No': 0}).astype(int)
    df['dependents_encoded'] = df.dependents.map({'Yes': 1, 'No': 0}).astype(int)
    df['phone_service_encoded'] = df.phone_service.map({'Yes': 1, 'No': 0}).astype(int)
    df['paperless_billing_encoded'] = df.paperless_billing.map({'Yes': 1, 'No': 0}).astype(int)
    df['churn_encoded'] = df.churn.map({'Yes': 1, 'No': 0}).astype(int)
    df['senior_citizen_encoded'] = df.senior_citizen.map({'Yes': 1, 'No': 0}).astype(int)
    
    

    # Get dummies for non-binary categorical variables
    dummy_df = pd.get_dummies(df[['multiple_lines', \
                              'online_security', \
                              'online_backup', \
                              'device_protection', \
                              'tech_support', \
                              'streaming_tv', \
                              'streaming_movies', \
                              'contract_type', \
                              'internet_service_type', \
                              'payment_type']], dummy_na = False, \
                              drop_first = True)
    
    # Concatenate dummy df to original df
    df = pd.concat([df, dummy_df], axis = 1)

    #change from CamelCase to snake_case format when pulled from database
    column_list = list(df.columns)
    for i, col in enumerate(column_list):
        column_list[i] = inflection.underscore(column_list[i].replace(' ', '_'))
    
    # lowercase dummies    
    df.columns = column_list
    df.columns = df.columns.str.lower()

return df 